In [2]:
from langchain.llms.bedrock import Bedrock
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
import boto3
import botocore
config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)


bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

file_path = (r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\Bryckle\Dallas, TX Olympus, Master Lease Agreement, Brillio LLC.pdf")

loader = PyPDFLoader(file_path)
docs=loader.load()


In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=texts,embedding=embeddings)

template = """
    You will be given a query, Your task is to find an answer or give information about the query with respect to the document by performing a similarity search.
    Consider the following conditions,
    - If query is a topic, look for information or statements or sentences which are related to the query in the document.
    - The answer does not need to be specifically related, it can be loosely related as well.
    - If you find any statements that are directly related to the query, then explain those statements in easy or layman terms, so someone with no expertise in that field can understand.
    - If query is a question, understand the context of query and then look for similar statements or sentences in the document which also have the same context.
    - It is not necessary to return direct statements from the document as an answer. You can also return loosely related answers to the query.
    - If you cannot find any direct statements or directly relevant answers, do not return that you cannot find any direct statements. Then, you have to perform a semantic search instead of looking for exact words in the document, that is, understanding the context from the query and looking for something similar in the document.
    - It is not mandatory to look for direct statements, you can also look at statements with a similar meaning and context.
    - It is fine if you cannot find any directly related statements in the document. You can look for sentences with similar meaning and can also return loosely related answers.
    - Elaborate the answer as much as you can.
    - The output should just be a bullet list of points which has the summary of all points obtained from the search
    - If you cannot find a relevant answer, then perform a similarity search on all statements which have a similar meaning to the query, and the document.
    - If even after doing the search on similar statements, you can't find an answer, you can just say that you are not able to find any answer without saying anything else.

{context}
{question}
"""

In [4]:
retriever = db.as_retriever(search_type='similarity', search_kwargs={"k": 3})
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 30000})
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)

In [83]:
question="If email or phone number is provided for the tenant contact for notices, capture that"
result = qa.run(question)
print(result)

 The document discusses notices and contact information for the tenant in several places. Here are the key relevant statements I found:

1. If the Basic Lease Information section leaves tenant's notice information blank or incomplete, then prior to the Commencement Date, the registered agent address for the tenant shall be an appropriate address for notice. After the Commencement Date, the Premises shall be an appropriate address for notice.

2. There is a provision that the landlord can show the Premises to prospective purchasers, lenders, or new tenants by giving the tenant 10-days prior notice. Such notice may be delivered by email to Austin.hamilton@brillio.com. 

3. The document also grants the landlord a power of attorney to execute and file any financing statement or other instrument necessary to perfect a security interest, at the tenant's expense.

In summary, the key contact information provided for sending notices to the tenant is:

- Prior to Commencement Date: Registered a